In [ ]:
#%xmode Plain
from jeudetaquin import JeuDeTaquin
t = SkewTableau([[None,2,3],[None,4],[5]]) 

In [ ]:
from sage_combinat_widgets import GridViewWidget
from sage_combinat_widgets.grid_view_editor import get_adapter

In [ ]:
g = GridViewWidget(t, display_convention='fr')
g

In [ ]:
from sage_widget_adapters.combinat.skew_tableau_grid_view_adapter import SkewTableauGridViewAdapter
class JeuDeTaquinGridViewAdapter(SkewTableauGridViewAdapter):
    objclass = JeuDeTaquin
    constructorname = 'JeuDeTaquin'
    addablecelltype = bool
    addablecellzero = False

    def display_to_cell(self, display_value, display_type):
        if type(display_value) == bool:
            return display_value
        return super(JeuDeTaquinGridViewAdapter, self).display_to_cell(display_value, display_type)

    @staticmethod
    def addable_cells(obj):
        if obj._hole:
            return [obj._hole]
        return obj.inner_shape().corners()
    
    @staticmethod
    def removable_cells(obj):
        if not obj._hole:
            return []
        if obj._hole in obj.outer_shape().corners():
            return (obj._hole[0] - 1, obj._hole[1])
        else:
            return (obj._hole[0], obj._hole[1] + 1)

    @classmethod
    def set_cell(cls, obj, pos, val):
        if not obj._new_st:
            return super(JeuDeTaquinGridViewAdapter, cls).set_cell(obj, pos, val)
        for i in range(len(obj._new_st)):
            for j in range(len(obj._new_st[i])):
                if (i, j) == pos:
                    obj._new_st[i][j] = val
                    break
        return obj
        
    @classmethod
    def add_cell(cls, obj, pos, val, dirty={}):
        r'''
        create hole, then slide and return a new tab if possible, 
        otherwise the old tab with a `_new_st` list
        '''
        if not obj.has_hole():
            obj.create_hole(pos)
        assert pos == obj._hole # check before sliding
        obj.slide()
        try:
            new_obj.__init__(obj._new_st)
            new_obj._hole = obj._hole
        except:
            new_obj = obj
        return obj

    @staticmethod
    def compute_cells(obj):
        cells = SkewTableauGridViewAdapter.compute_cells(obj)
        if obj._hole:
            cells[obj._hole] = False
        if obj._new_st:
            for i in range(len(obj._new_st)):
                for j in range(len(obj._new_st[i])):
                    if obj._new_st[i][j] > 0:
                        cells[(i,j)] = obj._new_st[i][j]
                    elif (i,j) in cells:
                        del(cells[(i,j)])
        return cells

#t = SkewTableau([[None,2,3],[None,4],[5]])
t = SkewTableau([[None,None,None,4,4,5,6,7],[None,2,4,6,7,7,7],[None,4,5,8,8,9],[None,6,7,10],[None,8,8,11],[None],[4]])
jdt = JeuDeTaquin(t)
A = JeuDeTaquinGridViewAdapter()

In [ ]:
from sage_combinat_widgets.grid_view_widget import DisabledTextCell, AddableButtonCell#, textcell_layout
from ipywidgets import Layout
hole_layout = Layout(width='3em',height='2em', margin='0', padding='20px')

class JDTHoleWidget(AddableButtonCell):
    def __init__(self, position, layout=None, **kws):
        super(JDTHoleWidget, self).__init__(position, layout, **kws)
        self.description = '?'
        #print(self.position, self.value)
        #self.remove_class('addablebutton')
        #self.add_class('blankcell')

w = GridViewWidget(jdt, A, display_convention='fr', cell_widget_classes=[DisabledTextCell], addable_widget_class=JDTHoleWidget)

In [ ]:
# The player sees the initial jeu de taquin, with an clickable button for each possible hole.
w

In [ ]:
w.value